### Import the needed packages

In [ ]:
# package for using the the custom scripts 
import sys
sys.path.insert(0, "./../")
# package for handling files and directories
import os
# package for handling the training hyperparameter tracking
import wandb
# a custom package for handling the data
import utilities as utils
# package for handling the versioning of the models
from huggingface_hub import HfApi
# package for handling the environment variables
from dotenv import load_dotenv
# get the environment variables
load_dotenv()
# set the wandb api to silent
os.environ["WANDB_SILENT"] = "true"

### Global variable

In [ ]:
# store the path
path_full = os.environ.get("ORI_PATH")
path_dataset_ori = os.path.join(path_full, "datasets", "splitted", "original")
path_dataset_aug = os.path.join(path_full, "datasets", "splitted", "augmented")
path_model = os.path.join(path_full, "data", "model")
ori_path = {
    "train": os.path.join(path_dataset_ori, "train"),
    "val": os.path.join(path_dataset_ori, "val"),
    "test": os.path.join(path_dataset_ori, "test"),
}
aug_path = {
    "train": os.path.join(path_dataset_aug, "train"),
    "val": os.path.join(path_dataset_aug, "val"),
    "test": os.path.join(path_dataset_aug, "test"),
}
# the commit message
wandb_init_name = "retrain the model"
hf_commit_message = "retrain the model"
# the hyperparameter for the training
hyperparam = {
    "batch_size": 16,
    "epochs": 50,
    "img_size": 128,
    "shuffle": True,
    "model_type": "U-net",
    "metrics": ["accuracy", "MeanIoU", "Precision", "Recall"],
    "split_ratio": [7,1,2],
    "dataset_balance": True,
    "weighted_training": True,
}

### Import the images and mask

In [18]:
# get the original dataset
train_ori_img, train_ori_mask = utils.load_img_mask(ori_path["train"])
val_ori_img, val_ori_mask = utils.load_img_mask(ori_path["val"])
test_ori_img, test_ori_mask = utils.load_img_mask(ori_path["test"])

# get the augmented dataset
train_aug_img, train_aug_mask = utils.load_img_mask(aug_path["train"])
val_aug_img, val_aug_mask = utils.load_img_mask(aug_path["val"])
test_aug_img, test_aug_mask = utils.load_img_mask(aug_path["test"])

### Create the batched dataset

In [19]:
# create the tensor dataset for original dataset
trainset_ori = utils.create_dataset(train_ori_img, train_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_ori = utils.create_dataset(val_ori_img, val_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_ori = utils.create_dataset(test_ori_img, test_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])

# create the tensor dataset for augmented dataset
trainset_aug = utils.create_dataset(train_aug_img, train_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_aug = utils.create_dataset(val_aug_img, val_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_aug = utils.create_dataset(test_aug_img, test_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])

### Train model

In [ ]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_ori"
# train the model using original dataset
unet_custom, history, ev_result = utils.train_model(unet_custom,
                                                trainset_ori, valset_ori, testset_ori,
                                                path_model, file_model_name,
                                                hyperparam["epochs"])
# show the evaluation result
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/50
16/16 [==============================] - 39s 396ms/step - loss: 0.0095 - mean_px_acc: 0.4849 - auc: 0.7566 - precision: 0.5795 - recall: 0.4886 - val_loss: 0.0106 - val_mean_px_acc: 0.9842 - val_auc: 0.6782 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
16/16 [==============================] - 16s 221ms/step - loss: 0.0063 - mean_px_acc: 0.5654 - auc: 0.8882 - precision: 0.7562 - recall: 0.6227 - val_loss: 0.0109 - val_mean_px_acc: 0.9826 - val_auc: 0.5062 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
16/16 [==============================] - 13s 194ms/step - loss: 0.0052 - mean_px_acc: 0.6916 - auc: 0.9268 - precision: 0.8255 - recall: 0.6678 - val_loss: 0.0111 - val_mean_px_acc: 0.9825 - val_auc: 0.5057 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
16/16 [==============================] - 13s 199ms/step - loss: 0.0048 - mean_px_acc: 0.8163 - auc: 0.9399 - precision: 0.8503 - recall: 0.7269 - val_loss: 0.0116 - val_mean_px

In [ ]:
# initiate the huggingface api 
api = HfApi()
repo = api.create_repo("CAD-Glaucoma_Segmentation", repo_type="model", exist_ok=True)

In [ ]:
# push the model artifact and hyperparameter to wandb
## set the hyperparameter for the wandb
hyperparam["aug"] = False
## initiate the project
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"ori - {wandb_init_name}",
            config=hyperparam)
## write the training history and validation result to wandb log
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["mean_px_acc"][epoch],
        "train_auc": history.history["auc"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "train_f1": 2 * (history.history["precision"][epoch] * history.history["recall"][epoch]) / (history.history["precision"][epoch] + history.history["recall"][epoch] + 1e-7),
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_mean_px_acc"][epoch],
        "val_auc": history.history["val_auc"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
        "val_f1": 2 * (history.history["val_precision"][epoch] * history.history["val_recall"][epoch]) / (history.history["val_precision"][epoch] + history.history["val_recall"][epoch] + 1e-7),
    })
## store the model artifact
wandb.log_artifact(os.path.join(path_model, f"{file_model_name}.h5"))
wandb.finish()

In [ ]:
# push the model to the huggingface hub
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join(path_model, f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=hf_commit_message,
)

unet_model_ori.h5: 100%|██████████| 1.57M/1.57M [00:01<00:00, 812kB/s] 


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation/commit/4020b4609c97767c477c2a02c924e4796d1f4707', commit_message='retrain the model', commit_description='', oid='4020b4609c97767c477c2a02c924e4796d1f4707', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/CAD-Glaucoma_Segmentation'), pr_revision=None, pr_num=None)

In [ ]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_aug"
# train the model using augmented dataset
unet_custom, history, ev_result = utils.train_model(unet_custom,
                                                trainset_aug, valset_aug, testset_aug,
                                                path_model, file_model_name,
                                                hyperparam["epochs"])
# show the evaluation result
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/50
16/16 [==============================] - 16s 236ms/step - loss: 0.0085 - mean_px_acc: 0.4246 - auc: 0.8003 - precision: 0.6547 - recall: 0.5531 - val_loss: 0.0101 - val_mean_px_acc: 0.8626 - val_auc: 0.8837 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
16/16 [==============================] - 3s 110ms/step - loss: 0.0061 - mean_px_acc: 0.5818 - auc: 0.8943 - precision: 0.7801 - recall: 0.6652 - val_loss: 0.0103 - val_mean_px_acc: 0.9779 - val_auc: 0.6684 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
16/16 [==============================] - 4s 112ms/step - loss: 0.0056 - mean_px_acc: 0.7158 - auc: 0.9159 - precision: 0.8096 - recall: 0.6867 - val_loss: 0.0109 - val_mean_px_acc: 0.9722 - val_auc: 0.5379 - val_precision: 1.0000 - val_recall: 0.0021
Epoch 4/50
16/16 [==============================] - 4s 106ms/step - loss: 0.0053 - mean_px_acc: 0.7848 - auc: 0.9224 - precision: 0.8213 - recall: 0.7232 - val_loss: 0.0117 - val_mean_px_acc: 0.974

In [ ]:
# push the model artifact and hyperparameter to wandb
## set the hyperparameter for the wandb
hyperparam["aug"] = True
## initiate the project
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"aug - {wandb_init_name}",
            config=hyperparam)
## write the training history and validation result to wandb log
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["mean_px_acc"][epoch],
        "train_auc": history.history["auc"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "train_f1": 2 * (history.history["precision"][epoch] * history.history["recall"][epoch]) / (history.history["precision"][epoch] + history.history["recall"][epoch] + 1e-7),
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_mean_px_acc"][epoch],
        "val_auc": history.history["val_auc"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
        "val_f1": 2 * (history.history["val_precision"][epoch] * history.history["val_recall"][epoch]) / (history.history["val_precision"][epoch] + history.history["val_recall"][epoch] + 1e-7),
    })
## store the model artifact
wandb.log_artifact(os.path.join(path_model, f"{file_model_name}.h5"))
wandb.finish()

In [ ]:
## push the model to the huggingface hub
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join(path_model, f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=f"aug - {hf_commit_message}",
)

unet_model_aug.h5: 100%|██████████| 1.57M/1.57M [00:02<00:00, 575kB/s] 


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation/commit/a9e6320f4c837eb81b76adb33774960afbf63957', commit_message='aug - retrain the model', commit_description='', oid='a9e6320f4c837eb81b76adb33774960afbf63957', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/CAD-Glaucoma_Segmentation'), pr_revision=None, pr_num=None)